In [5]:
import math 
import base64
import binascii
import textwrap

def XOR(base, other):
    final = ''
    for i in range(len(base)):
        if base[i] == other[i]:
            final += '0'
        else:
            final += '1'
    return final

def keygen(key):
    '''takes in a key, transfers it to a binary key of length 64 bits, creates a subkey of length 58 using PC-1, splits the subkey into 
    halves and creates 16 sets of halves built by shifting the previous pair to the left in a certain pattern.
    Halves are then rejoined and create pairs, which will then be used to create subkeys using PC-2'''

    # Initialising hex string 
    key = int(key, base = 16)
    key = bin(key)[2:]
    missing = 64-len(key)
    pad = '0'*missing
    key = pad+key
    
    #subkey conversion with PC-1
    K = {}
    for i in range(len(key)):
        K[i] = key[i]
    
    #PC-1 application
    Kprime = f"{K[56]}{K[48]}{K[40]}{K[32]}{K[24]}{K[16]}{K[8]}{K[0]}{K[57]}{K[49]}{K[41]}{K[33]}{K[25]}{K[17]}{K[9]}{K[1]}{K[58]}{K[50]}{K[42]}{K[34]}{K[26]}{K[18]}{K[10]}{K[2]}{K[59]}{K[51]}{K[43]}{K[35]}{K[62]}{K[54]}{K[46]}{K[38]}{K[30]}{K[22]}{K[14]}{K[6]}{K[61]}{K[53]}{K[45]}{K[37]}{K[29]}{K[21]}{K[13]}{K[5]}{K[60]}{K[52]}{K[44]}{K[36]}{K[28]}{K[20]}{K[12]}{K[4]}{K[27]}{K[19]}{K[11]}{K[3]}"
    
    
    C0 = Kprime[:28]
    D0 = Kprime[28:]
    
    #create sub pairs
    shifts = {
    2: 1,
    3: 2,
    4: 2,
    5: 2,
    6: 2,
    7: 2,
    8: 2,
    9: 1,
    10: 2,
    11: 2,
    12: 2,
    13: 2,
    14: 2,
    15: 2,
    16: 1
}
        
        
    C1 = C0[1:] + C0[0]
    D1 = D0[1:] + D0[0]
    pairs = {}
    pairs[1] = (C1, D1)    
        
    for i in shifts:    
        pastC = pairs[i-1][0]
        pastD = pairs[i-1][1]
        #changes how much is shifted depending on shifts
        if shifts[i] == 2:
            pairs[i] = (pastC[2:] + pastC[0] + pastC[1], pastD[2:] + pastD[0] + pastD[1])
        else:
            pairs[i] = (pastC[1:] + pastC[0], pastD[1:] + pastD[0])
        
    subkeys = []
    
    for pair in pairs.values():
        cd = pair[0]+pair[1]
        #pc-2
        subkeys.append(f"{cd[13]}{cd[16]}{cd[10]}{cd[23]}{cd[0]}{cd[4]}{cd[2]}{cd[27]}{cd[14]}{cd[5]}{cd[20]}{cd[9]}{cd[22]}{cd[18]}{cd[11]}{cd[3]}{cd[25]}{cd[7]}{cd[15]}{cd[6]}{cd[26]}{cd[19]}{cd[12]}{cd[1]}{cd[40]}{cd[51]}{cd[30]}{cd[36]}{cd[46]}{cd[54]}{cd[29]}{cd[39]}{cd[50]}{cd[44]}{cd[32]}{cd[47]}{cd[43]}{cd[48]}{cd[38]}{cd[55]}{cd[33]}{cd[52]}{cd[45]}{cd[41]}{cd[49]}{cd[35]}{cd[28]}{cd[31]}")

    return subkeys

    
def sboxes(Bs):
    '''creates 8 s-boxes, then rewrites 8 six-bit pieces of input as 4 bit outputs'''
    Ss = [[14, 4, 13, 1, 2, 15, 11, 8, 3, 10, 6, 12, 5, 9, 0, 7,
     0, 15, 7, 4, 14, 2, 13, 1, 10, 6, 12, 11, 9, 5, 3, 8,
     4, 1, 14, 8, 13, 6, 2, 11, 15, 12, 9, 7, 3, 10, 5, 0,
     15, 12, 8, 2, 4, 9, 1, 7, 5, 11, 3, 14, 10, 0, 6, 13], 
          
    [15, 1, 8, 14, 6, 11, 3, 4, 9, 7, 2, 13, 12, 0, 5, 10,
     3, 13, 4, 7, 15, 2, 8, 14, 12, 0, 1, 10, 6, 9, 11, 5,
     0, 14, 7, 11, 10, 4, 13, 1, 5, 8, 12, 6, 9, 3, 2, 15,
     13, 8, 10, 1, 3, 15, 4, 2, 11, 6, 7, 12, 0, 5, 14, 9],

    [10, 0, 9, 14, 6, 3, 15, 5, 1, 13, 12, 7, 11, 4, 2, 8,
     13, 7, 0, 9, 3, 4, 6, 10, 2, 8, 5, 14, 12, 11, 15, 1,
     13, 6, 4, 9, 8, 15, 3, 0, 11, 1, 2, 12, 5, 10, 14, 7,
     1, 10, 13, 0, 6, 9, 8, 7, 4, 15, 14, 3, 11, 5, 2, 12],

    [7, 13, 14, 3, 0, 6, 9, 10, 1, 2, 8, 5, 11, 12, 4, 15,
     13, 8, 11, 5, 6, 15, 0, 3, 4, 7, 2, 12, 1, 10, 14, 9,
     10, 6, 9, 0, 12, 11, 7, 13, 15, 1, 3, 14, 5, 2, 8, 4,
     3, 15, 0, 6, 10, 1, 13, 8, 9, 4, 5, 11, 12, 7, 2, 14],

    [2, 12, 4, 1, 7, 10, 11, 6, 8, 5, 3, 15, 13, 0, 14, 9,
     14, 11, 2, 12, 4, 7, 13, 1, 5, 0, 15, 10, 3, 9, 8, 6,
     4, 2, 1, 11, 10, 13, 7, 8, 15, 9, 12, 5, 6, 3, 0, 14,
     11, 8, 12, 7, 1, 14, 2, 13, 6, 15, 0, 9, 10, 4, 5, 3],

    [12, 1, 10, 15, 9, 2, 6, 8, 0, 13, 3, 4, 14, 7, 5, 11,
     10, 15, 4, 2, 7, 12, 9, 5, 6, 1, 13, 14, 0, 11, 3, 8,
     9, 14, 15, 5, 2, 8, 12, 3, 7, 0, 4, 10, 1, 13, 11, 6,
     4, 3, 2, 12, 9, 5, 15, 10, 11, 14, 1, 7, 6, 0, 8, 13],

    [4, 11, 2, 14, 15, 0, 8, 13, 3, 12, 9, 7, 5, 10, 6, 1,
     13, 0, 11, 7, 4, 9, 1, 10, 14, 3, 5, 12, 2, 15, 8, 6,
     1, 4, 11, 13, 12, 3, 7, 14, 10, 15, 6, 8, 0, 5, 9, 2,
     6, 11, 13, 8, 1, 4, 10, 7, 9, 5, 0, 15, 14, 2, 3, 12],

    [13, 2, 8, 4, 6, 15, 11, 1, 10, 9, 3, 14, 5, 0, 12, 7,
     1, 15, 13, 8, 10, 3, 7, 4, 12, 5, 6, 11, 0, 14, 9, 2,
     7, 11, 4, 1, 9, 12, 14, 2, 0, 6, 10, 13, 15, 3, 5, 8,
     2, 1, 14, 7, 4, 10, 8, 13, 15, 12, 9, 0, 3, 5, 6, 11]
]
    sgrids = []
    
    for S in Ss:
        sgrid = {}
        row = 0
        col = 0

        for i in range(len(S)):
            pos = (row, col)
            sgrid[pos] = S[i]

            if col == 15:
                col = 0
                row += 1
            else:
                col += 1
        sgrids.append(sgrid)
    
    
    outs = []
    
    for i in range(len(Bs)):
        B = Bs[i]
        sgrid = sgrids[i]
        
        first = int(B[0]+B[-1], base = 2)
        second = int(B[1:5], base = 2)
              
        out = bin(sgrid[(first, second)])[2:]
        missing = 4-len(out)
        pad = '0'*missing
        out = pad+out
        outs.append(out)
        
    return outs
    
       
def subhalf_encoding(pastl, pastr, key):
    '''takes in a set of halves and creates two new ones with a given formula Ln = Rn-1, Rn = Ln-1 XOR f(Rn-1, Kn) where f(x) expands 32-bit
    input x to 48 bits and XORs it with a key'''
    #define Ln
    Ln = pastr
    
    #define Rn
    E = f"{pastr[31]}{pastr[0]}{pastr[1]}{pastr[2]}{pastr[3]}{pastr[4]}{pastr[3]}{pastr[4]}{pastr[5]}{pastr[6]}{pastr[7]}{pastr[8]}{pastr[7]}{pastr[8]}{pastr[9]}{pastr[10]}{pastr[11]}{pastr[12]}{pastr[11]}{pastr[12]}{pastr[13]}{pastr[14]}{pastr[15]}{pastr[16]}{pastr[15]}{pastr[16]}{pastr[17]}{pastr[18]}{pastr[19]}{pastr[20]}{pastr[19]}{pastr[20]}{pastr[21]}{pastr[22]}{pastr[23]}{pastr[24]}{pastr[23]}{pastr[24]}{pastr[25]}{pastr[26]}{pastr[27]}{pastr[28]}{pastr[27]}{pastr[28]}{pastr[29]}{pastr[30]}{pastr[31]}{pastr[0]}"
    
    XORedR = XOR(key, E)
    Bs = []
    count = 1
    current = ''
    
    for i in XORedR:
        
        current += i
        
        if count == 6:
            Bs.append(current)
            current = ''
            count = 0
        count += 1
    
    Bs = sboxes(Bs)
    unpermed = ''
    for i in Bs:
        unpermed += i
    
    P = f"{unpermed[15]}{unpermed[6]}{unpermed[19]}{unpermed[20]}{unpermed[28]}{unpermed[11]}{unpermed[27]}{unpermed[16]}{unpermed[0]}{unpermed[14]}{unpermed[22]}{unpermed[25]}{unpermed[4]}{unpermed[17]}{unpermed[30]}{unpermed[9]}{unpermed[1]}{unpermed[7]}{unpermed[23]}{unpermed[13]}{unpermed[31]}{unpermed[26]}{unpermed[2]}{unpermed[8]}{unpermed[18]}{unpermed[12]}{unpermed[29]}{unpermed[5]}{unpermed[21]}{unpermed[10]}{unpermed[3]}{unpermed[24]}"
    
    Rn = XOR(pastl, P)
    
    return (Ln, Rn)
    
    
def encode(M, keys):
    '''encodes a plaintext message M using keys'''
    # Initialising hex string
    M = int(M, base = 16)
    M = bin(M)[2:]
    missing = 64-len(M)
    pad = '0'*missing
    M = pad+M
    
    #IP (initial permutation) step
    IP = f"{M[57]}{M[49]}{M[41]}{M[33]}{M[25]}{M[17]}{M[9]}{M[1]}{M[59]}{M[51]}{M[43]}{M[35]}{M[27]}{M[19]}{M[11]}{M[3]}{M[61]}{M[53]}{M[45]}{M[37]}{M[29]}{M[21]}{M[13]}{M[5]}{M[63]}{M[55]}{M[47]}{M[39]}{M[31]}{M[23]}{M[15]}{M[7]}{M[56]}{M[48]}{M[40]}{M[32]}{M[24]}{M[16]}{M[8]}{M[0]}{M[58]}{M[50]}{M[42]}{M[34]}{M[26]}{M[18]}{M[10]}{M[2]}{M[60]}{M[52]}{M[44]}{M[36]}{M[28]}{M[20]}{M[12]}{M[4]}{M[62]}{M[54]}{M[46]}{M[38]}{M[30]}{M[22]}{M[14]}{M[6]}"
    
    #splitting into halves and generating pairs
    L0 = IP[:32]
    R0 = IP[32:]
    pairs = {}
    pairs[0] = (L0, R0)
    
    for i in range(1, 17):
        L = pairs[i-1][0]
        R = pairs[i-1][1]
        pairs[i] = subhalf_encoding(L, R, keys[i-1])
    
    R16L16 = pairs[16][1]+pairs[16][0]
    
    
    final = f"{R16L16[39]}{R16L16[7]}{R16L16[47]}{R16L16[15]}{R16L16[55]}{R16L16[23]}{R16L16[63]}{R16L16[31]}{R16L16[38]}{R16L16[6]}{R16L16[46]}{R16L16[14]}{R16L16[54]}{R16L16[22]}{R16L16[62]}{R16L16[30]}{R16L16[37]}{R16L16[5]}{R16L16[45]}{R16L16[13]}{R16L16[53]}{R16L16[21]}{R16L16[61]}{R16L16[29]}{R16L16[36]}{R16L16[4]}{R16L16[44]}{R16L16[12]}{R16L16[52]}{R16L16[20]}{R16L16[60]}{R16L16[28]}{R16L16[35]}{R16L16[3]}{R16L16[43]}{R16L16[11]}{R16L16[51]}{R16L16[19]}{R16L16[59]}{R16L16[27]}{R16L16[34]}{R16L16[2]}{R16L16[42]}{R16L16[10]}{R16L16[50]}{R16L16[18]}{R16L16[58]}{R16L16[26]}{R16L16[33]}{R16L16[1]}{R16L16[41]}{R16L16[9]}{R16L16[49]}{R16L16[17]}{R16L16[57]}{R16L16[25]}{R16L16[32]}{R16L16[0]}{R16L16[40]}{R16L16[8]}{R16L16[48]}{R16L16[16]}{R16L16[56]}{R16L16[24]}"

    int_final = int(final, 2)

    hex_final = format(int_final, 'X')

    return hex_final


def decrypt(message, keys):
    '''decodes a DES-encoded message by reversing keys and rerunning DES'''
    keys.reverse()
    out = encode(message, keys)
    return out


flag = True
while flag:
    valid = True
    key = input("input a key in base 16 (numbers 0-9, letters A-F). The input must be 16 bits long.")
    if len(key) != 16:
        print(f'invalid. key is {len(key)} bits long.')
        valid = False
    for i in key:
        if i not in "1234567890ABCDEF":
            print('invalid response.')
            valid = False
    if valid == True:
        flag = False

    
keys = keygen(key)

message = input("input a message to encode using DES")
    
    
#base 16 encoder and data splitter
messages = textwrap.wrap(base64.b16encode(message.encode('utf-8')).decode('utf-8'), 16)

final = ''
finalplain = ''

for m in messages:
    ciphertext = encode(m, keys)
    decr = decrypt(ciphertext, keys)
    
    final += ciphertext
    finalplain += decr
    
    
print('ciphertext:', final)

outbytes = base64.b16decode(finalplain)
out = outbytes.decode('utf-8')

print('decryption:', out)

input a key in base 16 (numbers 0-9, letters A-F). The input must be 16 bits long. AADE223389987CCD

input a message to encode using DES Thank You!

ciphertext: 3818E0B22244FAA23F1E6FC46EE09A73
decryption: Thank You!


In [66]:
import base64
import textwrap
data_to_encode = "Hello, World!"

# Encode the data
encoded_data = base64.b16encode(data_to_encode.encode('utf-8'))
print(textwrap.wrap(encoded_data.decode('utf-8'), 16))
# Print the encoded data
print(encoded_data.decode('utf-8'))

db = base64.b16decode(encoded_data)
print(db.decode('utf-8'))

b'3M\xbd\x153)\xc2\xbf\xef\xa5\xec\xa3\xaf.j\x854\xb2\x89\x92\xa6\x9b\xae\xf6\xad\xd3\x81D\xac\xcd\xaf\x06\xe4g\xf7 \x9e$A\xbfm,+\xf9\xf33\xc2!\x87\xe6\x10Ar\x13\xa9\xca\x0cl\xfc!\xe1t\xf6\xd2J\xf0\xc0\x1e\x17\x84<\x89\x02\xc8\x90P\x080\xf9"n\xa7\xff:\x1f\xda\xa0\xb0'


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xbd in position 2: invalid start byte

In [9]:
def decrypt(message):
    keys.reverse()
    out = encode(message)
    return out

decrypt('C1F5F89DF2D3B188')

0000000000000000000000000000101011011110111111100100010100110010


'ADEFE4532'

0